In [ ]:
#hide
#all_wip

# 🧠 Introduction

Hey all, this competition is going to be an interesting one given the nature of the dataset. It's not a straighforward image classification problem and one can forulate the problem statement in multiple ways.

In my [EDA kernel](http://wandb.me/brain-eda), I have shown that the MRI images per patient and scan type is sequential in nature. 

As a doctor I would be interested to look at multiple slices (images) of the MRI Sequence to determine if a patient has brain tumor or not. Thus I have formulated the brain tumor classification problem statement as Video Classification. 

🙏 This is a work in progress and I would love to hear your suggestions to improve the training pipeline. 

I am using the dataset created by [Jonathan Besomi](https://www.kaggle.com/jonathanbesomi). Many thanks to him for creating this. You can find the data [here](https://www.kaggle.com/jonathanbesomi/rsna-miccai-png). 

I have implemented the pipeline in TensorFlow and using [Weights and Biases](https://wandb.ai/site) for experiment tracking and data visualization. 

# 🔭 Imports and Setup

In [ ]:
#export
import os
import re
import gc
import glob
import imageio
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print('TF version: ', tf.__version__)
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from sklearn.model_selection import train_test_split

TF version:  2.4.1


Set up Weights and Biases

In [ ]:
#export
import wandb
print('W&B version: ', wandb.__version__)
from wandb.keras import WandbCallback

wandb.login()

W&B version:  0.10.33


wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#export
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


# 🌈 Prepare Dataset

There are four sub-directories per patient corresponding to different MRI Image Sequencing methods. In this kernel, I am using "FLAIR" MRI to get the balls rolling. To get the maximum out of the dataset using every sequencing method is recommended. 

In [ ]:
# Load training csv file
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')

def get_patient_id(patient_id):
    if patient_id < 10:
        return '0000'+str(patient_id)
    elif patient_id >= 10 and patient_id < 100:
        return '000'+str(patient_id)
    elif patient_id >= 100 and patient_id < 1000:
        return '00'+str(patient_id)
    else:
        return '0'+str(patient_id)

def get_path(row):
    patient_id = get_patient_id(row.BraTS21ID)
    return f'../input/rsna-miccai-png/train/{patient_id}/FLAIR/'

df['path'] = df.apply(lambda row: get_path(row), axis=1)

# Removing two patient ids from the dataframe since there are not FLAIR directories for these ids. 
df = df.loc[df.BraTS21ID!=109]
df = df.loc[df.BraTS21ID!=709]
df = df.reset_index(drop=True)

df.head()

,BraTS21ID,MGMT_value,path
0,0,1,../input/rsna-miccai-png/train/00000/FLAIR/
1,2,1,../input/rsna-miccai-png/train/00002/FLAIR/
2,3,0,../input/rsna-miccai-png/train/00003/FLAIR/
3,5,1,../input/rsna-miccai-png/train/00005/FLAIR/
4,6,1,../input/rsna-miccai-png/train/00006/FLAIR/


Prepare train-test split. Note that there are only 585 patients so if you are doing video classification, K-fold training might be beneficial. 

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.1, stratify=df.MGMT_value.values)
print(f'Size of train_df: {len(train_df)}; valid_df: {len(valid_df)}')

Size of train_df: 524; valid_df: 59


In [ ]:
CONFIG = dict(
    NUM_FRAMES = 10,
    BATCH_SIZE = 8,
    EPOCHS = 100,
    IMG_SIZE = 224,
    LSTM_UNITS = 256,
    competition = 'rsna-miccai-brain',
    _wandb_kernel = 'ayut'
)

# 🚀 Video Classification Data Pipeline

A video classification data pipeline will compromise of multiple frames of the same video batched together. In order to batch the frames, the number of frames should be same. In this kerel it's controlled by `NUM_FRAMES`. 

In have implemented the data pipeline using purely `tf.data`. Here are the important points to note:

* The images in each `patient_id/FLAIR` directory is listed down using `glob.glob`. <br>
* The path to images need to be sorted as per the image id given by `Image-X.png`. This is done by `sorted_nicely` function below. <br>
* We need to select a window of frames given by `NUM_FRAMES`. I am using uniform sampling to do so. One can device a better sampling method. <br>
* Iterate through each frame (image), load them and resize them. 🚀 

In [ ]:
# https://stackoverflow.com/a/2669120/7636462
def sorted_nicely(l): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [ ]:
def decode_image(image):
    # convert the compressed string to a 3D uint8 tensor
    image = tf.image.decode_png(image, channels=1)
    # Normalize image
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    
    return image

def parse_frames(dirname):
    # get MRI images file paths for given patient 
    paths = glob.glob(dirname.decode('utf8')+'/*.png')
    # Sort the images to get sequential imaging
    paths = sorted_nicely(paths)
    
    # randomly select a window of images to be used as sequence
    start = tf.random.uniform((1,), maxval=len(paths)-CONFIG['NUM_FRAMES'], dtype=tf.int32)

    paths = tf.slice(paths, start, [CONFIG['NUM_FRAMES']])
    
    def get_frames(path):
        # Load image
        image = tf.io.read_file(path)
        image = decode_image(image)
        # Resize image
        image = tf.image.resize(image, (CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']))
        
        return image

    mri_images = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn=get_frames, elems=paths, fn_output_signature=tf.float32))
    
    return mri_images
    
def load_frame(df_dict):
    dirname = df_dict['path']
    paths = tf.numpy_function(parse_frames, [dirname], tf.float32)
    
    # Parse label
    label = df_dict['MGMT_value']
    label = tf.cast(label, tf.float32)
    
    return paths, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

trainloader = tf.data.Dataset.from_tensor_slices(dict(train_df))
validloader = tf.data.Dataset.from_tensor_slices(dict(valid_df))


trainloader = (
    trainloader
    .shuffle(1024)
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(CONFIG['BATCH_SIZE'])
    .prefetch(AUTOTUNE)
)

validloader = (
    validloader
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(CONFIG['BATCH_SIZE'])
    .prefetch(AUTOTUNE)
)

In [ ]:
# test out the trainloader
frames, labels = next(iter(trainloader))

In order to visualize the samples from our trainloader, I am using W&B. I find it easier to log everything onto W&B to visualize data than to write Matplotlib code. 

In [ ]:
run = wandb.init(project='brain-tumor-video', job_type='dataloader-viz')

os.makedirs('gifs/')
for i, frame in enumerate(frames):
    imageio.mimsave(f'gifs/out_{i}.gif', (frame*255).numpy().astype('uint8'))    

wandb.log({'examples': [wandb.Image(f'gifs/out_{i}.gif', caption=f'{label.numpy()}') for i, label in enumerate(labels)]})
    
run.finish()

wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


_runtime,7
_timestamp,1626905320
_step,0


_runtime,▁
_timestamp,▁
_step,▁


MRI Sequences, where each sequence is `NUM_FRAMES` long. 

![img](https://i.imgur.com/pKc7rnT.gif)

# 🚜 Model

In order to model both spatial and temporal nature of videos, we can use a hybrid of CNN + LSTM model. 

* The `FeatureExtractor` model uses an EfficientNetB0 model as CNN backbone. It will be used to model the spatial aspect of videos. <br>
* The `MRIModel` uses a `TimeDistributed` layer that runs the `FeatureExtractor` `NUM_FRAMES` times to get a vector of `(NUM_FRAMES, 1280)`. <br>
* This is then fed to a single LSTM layer. You can use GRU and even Transformer in place of LSTM. I have used 256 units as it gave me the best results. 

In [ ]:
def FeatureExtractor():
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainabe = True

    inputs = Input((CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], 1))
    x = Conv2D(3, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
    x = base_model(x, training=True)
    flattened_output = GlobalAveragePooling2D()(x)
    
    return Model(inputs, flattened_output)

tf.keras.backend.clear_session()
model = FeatureExtractor()
model.summary()

16711680/16705208 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 3)       30        
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
Total params: 4,049,601
Trainable params: 4,007,578
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
def MRIModel():
    inputs = Input((CONFIG['NUM_FRAMES'], CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], 1))
    feature_extractor = FeatureExtractor()
    
    time_wrapper = TimeDistributed(feature_extractor)(inputs)
    
    lstm_out = LSTM(CONFIG['LSTM_UNITS'], return_sequences=True, name="lstm")(time_wrapper)
    outputs = Dense(1, activation='sigmoid', name="lstm_sigmoid")(lstm_out)
    
    return Model(inputs, outputs)

tf.keras.backend.clear_session() 
model = MRIModel()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 224, 224, 1)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 1280)          4049601   
_________________________________________________________________
lstm (LSTM)                  (None, 10, 256)           1573888   
_________________________________________________________________
lstm_sigmoid (Dense)         (None, 10, 1)             257       
Total params: 5,623,746
Trainable params: 5,581,723
Non-trainable params: 42,023
_________________________________________________________________


# 🚅 Train

This is a simple training pipeline that uses early stopping as regularizer and `WandbCallback` to log the metrics to Weights and Biases.

In [ ]:
# Callbacks
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=0, mode='min',
    restore_best_weights=True
)

In [ ]:
tf.keras.backend.clear_session() 
model = MRIModel()
model.compile('adam', 'binary_crossentropy', metrics=['acc'])

run = wandb.init(project='brain-tumor-video', 
                 group='EffnetB0-LSTM-256', 
                 job_type='train', 
                 config=CONFIG)

# Train
_ = model.fit(trainloader, 
              epochs=CONFIG['EPOCHS'],
              validation_data=validloader,
              callbacks=[WandbCallback(),
                         earlystopper])

# Evaluate
loss, acc = model.evaluate(validloader)
wandb.log({'Val Accuracy': round(acc, 3)})

run.finish()

wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/100
66/66 [==============================] - 92s 1s/step - loss: 0.7761 - acc: 0.4861 - val_loss: 0.7122 - val_acc: 0.5203
Epoch 2/100
66/66 [==============================] - 63s 945ms/step - loss: 0.7013 - acc: 0.5174 - val_loss: 0.6994 - val_acc: 0.4881
Epoch 3/100
66/66 [==============================] - 61s 925ms/step - loss: 0.6993 - acc: 0.4932 - val_loss: 0.6985 - val_acc: 0.5119
Epoch 4/100
66/66 [==============================] - 65s 975ms/step - loss: 0.6951 - acc: 0.5142 - val_loss: 0.6841 - val_acc: 0.5305
Epoch 5/100
66/66 [==============================] - 63s 946ms/step - loss: 0.6890 - acc: 0.5576 - val_loss: 0.6878 - val_acc: 0.5220
Epoch 6/100
66/66 [==============================] - 59s 894ms/step - loss: 0.6911 - acc: 0.5363 - val_loss: 0.7056 - val_acc: 0.4695
Epoch 7/100
66/66 [==============================] - 58s 875ms/step - loss: 0.6766 - acc: 0.5938 - val_loss: 0.6829 - val_acc: 0.5831
Epoch 8/100
66/66 [==============================] - 58s 868ms/st

epoch,16
loss,0.68594
acc,0.54561
val_loss,0.67968
val_acc,0.56949
_runtime,1045
_timestamp,1626907212
_step,17
best_val_loss,0.67529
best_epoch,11
Val Accuracy,0.607


epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
loss,█▄▃▂▂▁▂▁▁▁▃▂▁▂▂▂▁
acc,▁▂▁▅▇▇▇█▆▇▃▃█▄▄▃▆
val_loss,█▆▅▃▃▇▂▆▆▃▅▁█▃▂▅▂
val_acc,▅▄▅▆▅▃█▄▃█▄█▁▆█▆▇
_runtime,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇███
_timestamp,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇███
_step,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
Val Accuracy,▁


### [Check out the W&B Dashboard $\rightarrow$](https://wandb.ai/ayush-thakur/brain-tumor-video?workspace=user-ayush-thakur)

![img](https://i.imgur.com/GyP8XNR.gif)

I have tried out three different LSTM units - 128, 256 and 512. 

You can see that the training curve is not stable. This kernel is a work in progress but wanted to share the idea with wider audience and see if it's a feasible idea to pursue. 

# WORK IN PROGRESS